# Consolidated workflow (continued)

Import initial libraries and data file. N.B. The data being used read in here is already numercised for use in sklearn's decision trees and random forests

## Tanzania - data analysis

In [13]:
# import libraries and data
import numpy as np
import pandas as pd
import matplotlib as plot
import seaborn as sns

# check select_data
df = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/all_numeric201808292240.csv')
df.shape

(59400, 24)

In [14]:
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

Sklearn's decision trees and random forests need categorical data changed into integers. See, e.g,: https://stackoverflow.com/questions/38108832/passing-categorical-data-to-sklearn-decision-tree

The above data set has already had this done.

In [15]:
df.columns

Index(['id', 'date_recorded', 'season_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'region_code', 'population',
       'public_meeting', 'scheme_management', 'permit', 'wpt_age',
       'construction_year', 'extraction_type_group', 'management_group',
       'payment_type', 'water_quality', 'quantity_group', 'source_type',
       'source_class', 'waterpoint_type_group', 'status_group'],
      dtype='object')

In [16]:
chosen_predictors = ['season_recorded', 'gps_height', 'installer', 'basin', 'region_code', 'population',
                     'public_meeting', 'scheme_management', 'permit', 'wpt_age','construction_year', 
                     'extraction_type_group', 'management_group', 'payment_type', 'water_quality', 
                     'quantity_group', 'source_type', 'waterpoint_type_group'
                    ]

predictors = df[chosen_predictors]
targets = df['status_group']

# train/test split
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=.2)

print("pred_train.shape:", pred_train.shape)
print("pred_test.shape:", pred_test.shape)
print("tar_train.shape:", tar_train.shape)
print("tar_test.shape:", tar_test.shape)

pred_train.shape: (47520, 18)
pred_test.shape: (11880, 18)
tar_train.shape: (47520,)
tar_test.shape: (11880,)


Build a classifier model with a decision tree, on train set

In [17]:
# Build model on training data; initiate classifier from sklearn, then fit it with the training data
classifier=DecisionTreeClassifier(random_state=2)
classifier=classifier.fit(pred_train,tar_train)

# predict for the test values and create confusion matrix
predictions=classifier.predict(pred_test)
print("Confusion matrix:")
print(sklearn.metrics.confusion_matrix(tar_test,predictions))
print("Accuracy score:")
print(sklearn.metrics.accuracy_score(tar_test, predictions))

Confusion matrix:
[[5181  333  922]
 [ 358  352  174]
 [ 973  179 3408]]
Accuracy score:
0.7526094276094276


Trying again, to play with parameters, experiment, etc.

In [18]:
# Playing with hyper-parameters. Also have varied the wording of the code from that in the MOOC. 
# (Just to check all in order with exceptionally high result.)
# train/test split
from sklearn.utils import shuffle
df = shuffle(df)
X_train, X_test, y_train, y_test = train_test_split(predictors[:], targets[:], test_size=.2)

# Build model on training data; initiate classifier from sklearn, then fit it with the training data
model = DecisionTreeClassifier(random_state=2)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.7587542087542087

For confusion matrix, N.B. that:

functional = 0

functional_needs_repair = 1

non-functional = 2

In [19]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Pred functional', 'Predicted func_need_repair', 'Predicted non_func'],
    index=['True functional', 'True func_need_repair', 'True non_func']
)

,Pred functional,Predicted func_need_repair,Predicted non_func
True functional,5174,362,898
True func_need_repair,374,341,161
True non_func,884,187,3499


In [20]:
# reprint for convenience
accuracy_score(y_test, y_predict)

0.7587542087542087

Key ingredient: see which features are most important

In [21]:
importance = (dict(zip(X_train, model.feature_importances_)))
sorted_items = sorted(importance.items(), key = lambda x: x[1], reverse=True)
sorted_items

[('gps_height', 0.2148415552395474),
 ('quantity_group', 0.1598393469477954),
 ('wpt_age', 0.15566434488960595),
 ('waterpoint_type_group', 0.08001933388124041),
 ('installer', 0.058229875583854625),
 ('region_code', 0.043238677823751376),
 ('payment_type', 0.04214092905112355),
 ('population', 0.035287835780893395),
 ('extraction_type_group', 0.03293399359535072),
 ('construction_year', 0.03139424756434459),
 ('scheme_management', 0.02924083122392693),
 ('source_type', 0.02657987369456549),
 ('basin', 0.025811997885999065),
 ('water_quality', 0.017220878624080286),
 ('season_recorded', 0.014300417523737617),
 ('permit', 0.013912331654398468),
 ('public_meeting', 0.010669524187098139),
 ('management_group', 0.008674004848686783)]

## Visualise the Decision Tree

It turns out that visualising the decision tree is quite straight forward.

There are two things one can initially do: limit or not limit maximum depth as folling example:
model = DecisionTreeClassifier(max_depth=None)

As below, push the image out to a tree.dot file. Find it on your computer amd then cut and paste the code into a web viewer/converter. See, e.g.:
http://www.ilovefreesoftware.com/03/featured/free-online-dot-to-png-converter-websites.html https://dreampuf.github.io/GraphvizOnline/

Right-clicking the image in GraphvizOnline, I could save the png to desktop.

If you limit maximum depth, your decision tree may be small enough to fit on one page. Else, cut and paste a number of lines of code from the tree.dot file, making sure you finish as follows in the online converter, e.g.:

...
44 -> 54 ; 
}

If you do limit the max depth the tree will be different, of course, than a full tree.

In [22]:
from sklearn import tree

# visualise
tree.export_graphviz(model, out_file='tree.dot') 
# cut and paste the tree.dot file info into webgraphviz.com in a browser. 
# (You can paste as many layers of the tree as you want. See notes above.)

In [23]:
# save to a file
# df.to_csv('/Users/RAhmed/data store/Wesleyan_Capstone/all_numeric201808292240.csv', index=False)

## Quick Aside - do KNN approach

In [24]:
## Import the Classifier.
from sklearn.neighbors import KNeighborsClassifier
## Instantiate the model with 5 neighbors. 
knn_list = []
for i in range(1,15):
    knn = KNeighborsClassifier(n_neighbors=i)
    ## Fit the model on the training data.
    knn.fit(X_train, y_train)
    ## See how the model performs on the test data.
    print("train set", knn.score(X_test, y_test))
    ## Append to list.
    knn_list.append({i: knn.score(X_test, y_test)})
knn_list

train set 0.675
train set 0.6765151515151515
train set 0.6904040404040404
train set 0.6858585858585858
train set 0.687962962962963
train set 0.688047138047138
train set 0.6867845117845118
train set 0.6856902356902357
train set 0.6856902356902357
train set 0.682996632996633
train set 0.6806397306397306
train set 0.6791245791245791
train set 0.6762626262626262
train set 0.6752525252525252


[{1: 0.675},
 {2: 0.6765151515151515},
 {3: 0.6904040404040404},
 {4: 0.6858585858585858},
 {5: 0.687962962962963},
 {6: 0.688047138047138},
 {7: 0.6867845117845118},
 {8: 0.6856902356902357},
 {9: 0.6856902356902357},
 {10: 0.682996632996633},
 {11: 0.6806397306397306},
 {12: 0.6791245791245791},
 {13: 0.6762626262626262},
 {14: 0.6752525252525252}]

## Do a global cross-validation approach on Decision Tree

https://chrisalbon.com/machine_learning/model_evaluation/cross-validaton/

In [25]:
# With k-fold cross validation. 
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

model = DecisionTreeClassifier(max_depth=None, random_state=2)

# Create k-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=2)

# Do k-fold cross-validation
cv_results = cross_val_score(model, # Classifier
                             predictors, # Feature matrix
                             targets, # Target vector
                             cv=kf, # Cross-validation technique
                             scoring="accuracy", # Loss function
                             n_jobs=-1) # Use all CPU scores

# Calculate mean
cv_results.mean()

0.7565824915824916

## Do a hold out cross-validation approach on Decision Tree

Two differences: 
> i). do cross-validation on train set only (not against all), and predict against test set<br>
> ii). experiment within the decision tree with various parameters

N.B. cross_validation does not give a fitted model. There is a cross_val_predict, but I think it doesn;t test against a seprate hold out set. (?)

In [26]:
# With k-fold cross validation. 
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score


# train/test split
X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=.2)

# Difference for this 
model = DecisionTreeClassifier(max_depth=20, criterion='gini', random_state=2, splitter='random')

# Create k-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Do k-fold cross-validation
cv_results = cross_val_score(model, # Classifier
                             X_train, # Feature matrix
                             y_train, # Target vector
                             cv=kf, # Cross-validation technique
                             scoring="accuracy", # Loss function
                             n_jobs=-1) # Use all CPU scores

# Calculate mean against train set!
print(cv_results.mean())

# So I have tweaked the 'model' (above) parameters to get the best model, 
# and can now test against the hold out set. ((Tweaked) Model has to be fitted)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)


0.766456228956229


0.767087542087542

^ above cross validation tweaking has improved accuracy (against hold out) by some 1.5%, great!

## Do Random Forest approach

In [27]:
# import all necessary libraries
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics
# feature importance
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier

/Users/RAhmed/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
# run Random Forest
from sklearn.ensemble import RandomForestClassifier
# classifier (sometimes use clf) is initiated
classifier=RandomForestClassifier(n_estimators=25, random_state=2)
# next step trains the model
classifier=classifier.fit(X_train,y_train)
# now we apply the classifier to the test data
predictions=classifier.predict(X_test)

# we look at confusion matrix and accuracy of prediction on test values
print("RANDOM FOREST")
print("Confusion matrix:")
print(sklearn.metrics.confusion_matrix(y_test,predictions))
print("Accuracy score:")
print(sklearn.metrics.accuracy_score(y_test, predictions))
print()
# display the relative importance of each attribute using RandomForestClassifier
# make this more readable by having the names of the predictors and having sorted
zipped = zip(predictors, classifier.feature_importances_)
my_list = list(zipped)
my_list.sort(key=lambda tup: tup[1], reverse=True)
print('RandomForestClassifier relative feature importance:')
for item in my_list:
    print('{0:42} {1:>42}'.format(item[0], item[1]))

# fit an Extra Trees model to the data (instead of Random Forest)
model = ExtraTreesClassifier(random_state=2)
model.fit(X_train,y_train)
predictions=model.predict(X_test)
print()
print("EXTRA TREE")
print("Confusion matrix:")
print(sklearn.metrics.confusion_matrix(y_test,predictions))
print("Accuracy score:")
print(sklearn.metrics.accuracy_score(y_test, predictions))
print()
# make this more readable by having the names of the predictors and having sorted
zipped = zip(predictors, model.feature_importances_)
my_list = list(zipped)
my_list.sort(key=lambda tup: tup[1], reverse=True)
print('ExtraTreesClassifier relative feature importance:')
for item in my_list:
    print('{0:42} {1:>42}'.format(item[0], item[1]))

RANDOM FOREST
Confusion matrix:
[[5518  205  641]
 [ 422  330  144]
 [ 897   93 3630]]
Accuracy score:
0.7978114478114479

RandomForestClassifier relative feature importance:
gps_height                                                        0.18247620155820027
wpt_age                                                           0.15007290078784227
quantity_group                                                    0.13048944016509997
construction_year                                                 0.07572882779816291
installer                                                        0.060412324176236994
waterpoint_type_group                                             0.05465688934128066
extraction_type_group                                             0.05057305527990822
payment_type                                                      0.04423999942365753
region_code                                                       0.04169789982828312
population                                         

## Do Random Forest with cross-validation and hold out set

In [29]:
# Random forest with cross-validation and hold out set
model = RandomForestClassifier(n_estimators=50, criterion='entropy', max_depth=18, 
                               bootstrap=True, oob_score=True, random_state=2)

# Create k-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Do k-fold cross-validation
cv_results = cross_val_score(model, # Classifier
                             X_train, # Feature matrix
                             y_train, # Target vector
                             cv=kf, # Cross-validation technique
                             scoring="accuracy", # Loss function
                             n_jobs=-1) # Use all CPU scores

# Calculate mean against train set!
cv_results.mean()

# Calculate mean against test set
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.807996632996633

In [30]:
# Find relative importance
zipped = zip(predictors, model.feature_importances_)
my_list = list(zipped)
my_list.sort(key=lambda tup: tup[1], reverse=True)
print('RandomForestClassifier relative feature importance:')
for item in my_list:
    print('{0:42} {1:>42}'.format(item[0], item[1]))

RandomForestClassifier relative feature importance:
quantity_group                                                     0.1465704225848066
wpt_age                                                           0.13350422537211326
gps_height                                                        0.12520558014004052
construction_year                                                 0.07520657346209542
installer                                                         0.06519399077410369
waterpoint_type_group                                             0.06322608823224003
extraction_type_group                                             0.05565311776848482
region_code                                                        0.0517810333050318
payment_type                                                     0.049226370242465484
population                                                        0.03847860262055452
basin                                                             0.03703656713355628
sc

## Naive Bayes Classifier

In [31]:
df2 = df
df2.head()

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
50475,33374,2013.075269,2,325,197,38.658823,-10.172128,7,7,3,...,1983,6,4,2,6,2,5,0,5,2
1923,7160,2013.061828,2,1159,323,33.535101,-2.047503,4,19,2,...,2012,1,4,6,7,4,4,1,1,2
398,38122,2013.123656,2,1342,157,34.098314,-2.887676,4,16,1,...,2001,5,4,2,6,1,5,0,3,2
9961,12797,2011.206989,1,291,157,36.164846,-8.932348,6,4,1,...,2010,5,4,1,6,1,5,0,3,0
26829,37333,2011.610215,0,1,769,33.232168,-2.870017,4,18,0,...,2009,5,0,2,6,1,5,0,3,0


In [32]:
predictors2 = df2[chosen_predictors]
targets2 = df2['status_group']

X_train, X_test, y_train, y_test = train_test_split(predictors2, targets2, test_size=.2)
X_train.head()

,season_recorded,gps_height,installer,basin,region_code,population,public_meeting,scheme_management,permit,wpt_age,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,waterpoint_type_group
54607,1,-21,720,8,5,4,1,7,0,12.212366,1999,10,4,5,0,1,4,1
48029,1,685,470,6,10,1,1,7,1,1.212366,2010,1,4,5,4,1,6,1
24246,1,64,223,6,0,0,1,7,1,36.209677,1975,4,4,6,6,0,0,1
1080,1,18,223,2,11,0,1,7,1,27.290323,1984,1,4,2,6,1,6,1
19754,0,0,544,4,18,0,1,7,1,2011.548387,0,10,4,2,6,0,0,1


In [33]:
df2['gps_height'].value_counts()[0]

4643

In [34]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
GaussianNB(priors=None)
y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.4707912457912458

### Run Logistic Regression

0 = func, 2 = non=func

Will use sklearn's methods, as will be easier to use with the way my data is set up for sklearn

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=.2)

# Standardize features 
# scaler = StandardScaler() 
# from sklearn.preprocessing import StandardScaler

logisticRegr = LogisticRegression(random_state=2, multi_class='ovr')
logisticRegr.fit(X_train, y_train)

# Use score method to get accuracy of model
score = logisticRegr.score(X_test, y_test)
print(score)


0.5925084175084175


## Results

Random seed set at 2 for all except Naive  Bayes Classifier and KNN (where n/a)

| Method | Cross-validation | Hold-out set | Accuracy |
| ---- | ---- | ---- | ---- |
| Naive Bayes | no | yes | 47.7% |
| Logistic regression | no | yes | 59.2% |
| KNN | no | yes | 68.1% |
| Decision tree | no | yes | 75.7% |
| Decision tree | yes | no | 75.7% |
| Decision tree | yes | yes | 76.3% |
| Extra tree | no | yes | 79.0% |
| Random forest | yes | yes | 81.2% |

## How results could be improved
> Frequency encoding of variables;<br>
> Nearest neighbour for construction year?;<br>
> Google Elevation API for missing gps_height;<br>
> Smaller train set, perhaps model is over-fitted?  (Look how DTree improved with hold out set, when using cross validation)<br>